In [56]:
from collections import Counter

In [303]:
from sklearn_hierarchical_classification.constants import ROOT

In [2]:
import pickle as pk
def save_obj(obj, name):
    """
    Saving the pickle object
    """
    with open(name + '.pkl', 'wb') as file:
        pk.dump(obj, file, pk.HIGHEST_PROTOCOL)


def load_obj(name):
    """
    loading the pickle object
    """
    with open(name + '.pkl', 'rb') as file:
        return pk.load(file)

In [88]:
yamtools = load_obj('/home/nnabizad/code/toolpred/data/yam/yam_tools')
ings = load_obj('/home/nnabizad/code/toolpred/data/yam/yam_ings')

In [89]:
yamtoolvocab = [l for j in yamtools for k in j for i in k for l in i.split()]

In [90]:
set(yamtoolvocab)

{'10-inch',
 '9-inch',
 '9x13-inch',
 '9x5-inch',
 'Dutch',
 'aluminum',
 'attachment',
 'bag',
 'bags',
 'baking',
 'baller',
 'baster',
 'bench',
 'blender',
 'board',
 'boiler',
 'bottomed',
 'bowl',
 'bowls',
 'brush',
 'bundt',
 'butter',
 'cake',
 'can',
 'casserole',
 'cast-iron',
 'cheesecloth',
 'chef’s',
 'coffee',
 'colander',
 'container',
 'cooker',
 'cookie',
 'cooling',
 'cup',
 'cupcake',
 'cups',
 'cutter',
 'cutting',
 'decorating',
 'dish',
 'double',
 'dough',
 'dutch',
 'duty',
 'extra-large',
 'flat',
 'flexible',
 'foil',
 'food',
 'fork',
 'form',
 'freezer',
 'frying',
 'gallon-sized',
 'garlic',
 'glass',
 'grater',
 'grill',
 'grinder',
 'hand',
 'heavy',
 'heavy-bottomed',
 'immersion',
 'jelly',
 'kitchen',
 'knife',
 'ladle',
 'large',
 'lid',
 'liners',
 'loaf',
 'lollipop',
 'mallet',
 'mandoline',
 'masher',
 'mat',
 'measuring',
 'meat',
 'medium',
 'melon',
 'metal',
 'microwave',
 'microwave-safe',
 'mill',
 'mini',
 'mitt',
 'mitts',
 'mixer',
 'mix

In [94]:
ingsvocab = [l for j in ings for k in j for i in k for l in i.split()]

In [95]:
set(ingsvocab)

{'&',
 '85%',
 'active',
 'adobo',
 'aidells®',
 'aioli',
 'all-purpose',
 'allspice',
 'almond',
 'and',
 'apple',
 'applesauce',
 'apricot',
 'asparagu',
 'au',
 'avocado',
 'baby',
 'bacon',
 'bagel',
 'baguette',
 'baking',
 'ball',
 'balsamic',
 'banana',
 'bar',
 'barbecue',
 'basil',
 'basmati',
 'bay',
 'bean',
 'beansprout',
 'beef',
 'bell',
 'berry',
 'bittersweet',
 'black',
 'blend',
 'blue',
 'blueberry',
 'boiling',
 'bone-in',
 'bone-in,',
 'boneless',
 'boston',
 'bread',
 'breadcrumb',
 'breakfast',
 'breast',
 'brick-style',
 'brisket',
 'broccoli',
 'broth',
 'brown',
 'brussels',
 'buffalo',
 'bulb',
 'bun',
 'butt',
 'butter',
 'buttermilk',
 'butternut',
 'butterscotch',
 'cabbage',
 'cajun',
 'candied',
 'candy',
 'cannellini',
 'canola',
 'caramel',
 'carrot',
 'cashew',
 'cauliflower',
 'cayenne',
 'celery',
 'center-cut',
 'chard',
 'cheddar',
 'cheese',
 'cherry',
 'chicken',
 'chile',
 'chili',
 'chili-garlic',
 'chily',
 'chip',
 'chipotle',
 'chive',
 'ch

In [99]:
import string as st
import re
def cleaing(txt):
    words = txt.split()
    for word in words:
        if word in st.punctuation: words.remove(word)
        if word == 'and': words.remove(word)
    txt = ' '.join(words)
    re.sub(r'([^\s\w-]|_)+', '', txt)
    return txt

In [122]:
allrec = []
for rec in ings:
    _rec = []
    for step in rec:
        _step =[]
        for _ing in step:
            _step.append(cleaing(_ing))
        _rec.append(set(_step))
    allrec.append(_rec)

In [123]:
allrec

[[{'long grain brown rice', 'salt', 'water'},
  {'garlic clove', 'medium carrot'},
  {'broccoli floret',
   'crushed red chily',
   'toasted sesame oil',
   'vegetable oil',
   'water'},
  {'fresno chile', 'green onion'}],
 [set(),
  {'nonstick cooking spray'},
  {'breakfast sausage link'},
  set(),
  set(),
  set(),
  set()],
 [{'unsalted butter'}, set(), set(), set(), set(), set()],
 [{'nonstick cooking spray'}, set(), set(), set(), set()],
 [{'garlic'}, set(), set(), set()],
 [{'nonstick cooking spray'},
  set(),
  set(),
  {'black pepper', 'italian parsley', 'salt'}],
 [{'red enchilada sauce'},
  {'black bean',
   'black pepper',
   'dried oregano',
   'ground coriander',
   'ground cumin',
   'mild diced hatch green chile',
   'olive oil',
   'salt'},
  set(),
  {'soft corn tortilla'},
  set(),
  set()],
 [{'honey',
   'light brown sugar',
   'rice vinegar',
   'sesame oil',
   'tamari soy sauce'},
  {'nonstick cooking spray'},
  set(),
  set(),
  {'scallion', 'sesame seed'}],
 [s

In [126]:
allings = [i for j in allrec for k in j for i in k]

In [127]:
set(allings)

{'85% lean ground beef',
 'active dry yeast',
 'aidells® smoked chicken sausage, chicken apple',
 'aioli',
 'all-purpose flour',
 'almond extract',
 'almond flour',
 'apple',
 'apple cider',
 'apple cider vinegar',
 'apple juice',
 'applesauce',
 'apricot jam',
 'asparagu',
 'au jus mix',
 'avocado',
 'avocado oil',
 'baby carrot',
 'baby kale',
 'baby potato',
 'baby spinach',
 'baby spinach leaf',
 'bacon',
 'bacon slice',
 'baguette',
 'baking apple',
 'baking powder',
 'baking soda',
 'ball park® bun size beef hot dog',
 'balsamic vinegar',
 'banana',
 'barbecue sauce',
 'basil',
 'basil pesto',
 'basmati rice',
 'bay leaf',
 'beansprout',
 'beef',
 'beef brisket',
 'beef broth',
 'beef cross-rib chuck roast',
 'beef tenderloin',
 'bell pepper',
 'bittersweet chocolate',
 'black bean',
 'black olife',
 'black pepper',
 'black peppercorn',
 'black sesame seed',
 'blue cheese',
 'blue cheese dressing',
 'blue tortilla chip',
 'boiling water',
 'bone-in pork chop',
 'bone-in, skin-on 

In [447]:
allmacparts = [i for j in macparts for k in j for i in k]

In [448]:
partset = set(allmacparts)
len(partset)

895

In [328]:
partset

{'0 phillips screw',
 '00 phillips screw',
 '1 phillips screw',
 '1.1mm 00 phillips screw',
 '1.1mm y00 tri-point screw',
 '1.2mm 00 phillips screw',
 '1.2mm phillips screw',
 '1.2mm silver phillips screw',
 '1.3mm phillips screw',
 '1.3mm silver phillips screw',
 '1.4mm phillips 0 screw',
 '1.4mm t5 torx screw',
 '1.4mm t8 torx screw',
 '1.4mm y0 tri-point screw',
 '1.4mm y0 tri-wing screw',
 '1.5mm 00 phillips screw',
 '1.5mm hex screw',
 '1.5mm phillips screw',
 '1.6mm phillips screw',
 '1.7mm phillips screw',
 '1.7mm t5 torx screw',
 '1.8mm p5 pentalobe screw',
 '1.9mm 00 phillips camera cable screw',
 '1.9mm phillips 0 screw',
 '1.9mm phillips screw',
 '10.3mm phillips screw',
 '10.3mm t8 torx screw',
 '10.5mm phillips screw',
 '10.8mm long hex bolt',
 '10.8mm long screw',
 '10mm 00 phillips screw',
 '10mm phillips screw',
 '10mm screw',
 '10mm t6 torx screw',
 '10mm t8 torx screw',
 '11.1mm t8 torx screw',
 '11.5mm screw',
 '11mm 00 phillips screw',
 '11mm phillips screw',
 '12.3

In [110]:
from collections import defaultdict

In [463]:
def hir_create(partset):
    class_hierarchy = defaultdict(list)
    for part in partset:
        words = part.split()
        if len(words)>1:
            if ' '.join(words[0:-1]) not in class_hierarchy[words[-1]]:
                class_hierarchy[words[-1]].append(' '.join(words[0:-1]))
        else:
            class_hierarchy["<ROOT>"].append(words[0])
    return class_hierarchy

In [464]:
def remove_ones(class_hierarchy, root):
    for tool in class_hierarchy.copy():
        if len(class_hierarchy[tool])==1 and class_hierarchy[tool][0]+ ' ' + tool not in class_hierarchy[root]:
            class_hierarchy[root].append(class_hierarchy[tool][0]+ ' ' + tool)
            del class_hierarchy[tool]
    return class_hierarchy

In [465]:
def next_level(tclass_hierarchy):
    for tool in list(tclass_hierarchy.keys()):
        for toolstring in tclass_hierarchy[tool][:]:
            words = toolstring.split()
            if len(words)>1:
                if words[-1] not in tclass_hierarchy[tool]:tclass_hierarchy[tool].append(words[-1])
                tclass_hierarchy[words[-1]].append(' '.join(words[0:-1]))
                tclass_hierarchy[tool].remove(toolstring)
        tclass_hierarchy = remove_ones(tclass_hierarchy, root=tool)
    return tclass_hierarchy

In [476]:
def check_dublicates(class_hierarchy):
    myset = set()
    for i in class_hierarchy:
        for j in class_hierarchy[i]:
            if j in myset:
                print(j)
            else:
                myset.add(j)

In [429]:
class_hierarchy = hir_create(partset)

In [430]:
class_hierarchy = remove_ones(class_hierarchy, root="<ROOT>")

In [433]:
class_hierarchy = next_level(class_hierarchy)

In [434]:
len(class_hierarchy)

75

In [435]:
len(class_hierarchy['screw'])

68

In [436]:
for i in class_hierarchy:
    print(i)
    print(class_hierarchy[i])
    print()

screw
['4.5mm', '5.3mm', 'black', '5.8mm', '3.8mm', '4mm', 'pentalobe', '2.2mm', '2.3mm', '4.9mm', 'torx', '8.6mm', '9.6mm', '5mm', '3.9mm', 'phillips', '4.3mm', '6mm', '5.4mm', '2.8mm', 'hex', '11.5mm', 't8', '2.1mm', 'tri-point', '7.8mm', 't6', '3.1mm', '3.4mm', '6.9mm', 'tri-wing', 'center', '10mm', '13mm', '6.5mm', 'light-colored', '6.2mm', 'long', 't5', '0', 'loop', 'shouldered', 'torque', 'pancake', 'standoff', 'silver', '8mm', 'cable', '3.3mm', '1', 'large-headed', 'battery', 't3', 'ground loop', 'phillips 3.4mm', 'shanked 13mm', 'top center', 't8 torque', '3.7mm pancake', 'right 8mm', '3.5mm 3.3mm', 'long silver phillips 1', '6.9mm large-headed', 't5 torx battery', '2.8mm t3', 'cosmetic', 'rubber', '4mm']

standoff
['5mm', '4mm', 'screw', 'secrews', 'torx', '14mm long 4mm']

grill
['hinge', 'plastic', 'rear plastic']

connector
['antenna', 'small', 'microphone', 'battery', 'trackpad', 'cable', 'fan', 'speaker', 'harddrive', 'subwoofer', 'black', 'opticaldrive', 'board', 'jack',

In [458]:
mactools = load_obj('/home/nnabizad/code/toolpred/data/mac/mac_tools')
macparts = load_obj('/home/nnabizad/code/toolpred/data/mac/mac_parts')
allmactools = [i for j in mactools for k in j for i in k]
toolset = set(allmactools)

In [466]:
toolclass_hierarchy = hir_create(toolset)

In [469]:
toolclass_hierarchy = remove_ones(toolclass_hierarchy, root="<ROOT>")

In [471]:
toolclass_hierarchy = next_level(toolclass_hierarchy)

In [478]:
toolclass_hierarchy

defaultdict(list,
            {'<ROOT>': ['tweezers',
              'pin',
              'multimeter',
              'iopener',
              'spudger',
              'screwdriver',
              'probe_and_pickset',
              'coin',
              'paperclip',
              'drill',
              'handle',
              'files',
              'air',
              'cloth',
              'scissors',
              'cable',
              'remover',
              'gloves',
              'striper',
              'pick',
              'dryer',
              'arcticlean',
              'wire',
              'wipes',
              'supply',
              'cards',
              'paste',
              'cleaner',
              'alcohol',
              'microscope',
              'tool',
              'pair',
              'programmer',
              'pliers',
              'clip',
              'gun',
              'suction handle',
              'diamond impregnated files',
              'co

In [473]:
def save_dic(dic, name):
    mydic = dict()
    for tool in dic:
        mydic[tool] = list(set(dic[tool]))
    save_obj(mydic, name)
    return mydic

In [477]:
check_dublicates(toolclass_hierarchy)

In [479]:
save_dic(toolclass_hierarchy, 'mactool_hi')

{'<ROOT>': ['tweezers',
  'supply',
  'suction handle',
  'pin',
  'scissors',
  'cards',
  'multimeter',
  'arcticlean',
  'diamond impregnated files',
  'iopener',
  'paste',
  'compressed air',
  'spudger',
  'cloth',
  'pick',
  'microfiber cleaning cloth',
  'usb cable',
  'utility scissors',
  'striper',
  'cable',
  'wipes',
  'microscope',
  'handle',
  'pliers',
  'screwdriver',
  'remover',
  'programmer',
  'adhesive remover',
  'disposable gloves',
  'wire striper',
  'ipad battery isolation pick',
  'hair dryer',
  'arctic silver arcticlean',
  'awg28 wire',
  'probe_and_pickset',
  'wire',
  'isopropyl alcohol wipes',
  'alcohol',
  'desktop power supply',
  'plastic cards',
  'gloves',
  'cleaner',
  'pair',
  'clip',
  'dryer',
  'arctic silver thermal paste',
  'ultra sonic cleaner',
  'coin',
  'gun',
  '99 isopropyl alcohol',
  'stereo microscope',
  'opening tool',
  'heavy-duty suction cups pair',
  'paperclip',
  'ftdi programmer',
  'needlenose pliers',
  'binder

In [406]:
save_dic(tclass_hierarchy)

{'screwdriver': ['1.5mm hex',
  'phillips',
  '2.5mm flathead',
  'hex',
  'pentalobe',
  'tri-point',
  'torx',
  'flathead'],
 'knife': ['precision utility', 'utility', 'tech', 'x-acto'],
 'torx': ['t4', 't10', 't3', 't6', 't8', 't5', 't9'],
 'tri-point': ['y00', 'y1', 'y0'],
 'pentalobe': ['p6', 'p5'],
 'phillips': ['ph00', 'ph000', 'ph0'],
 'nut': ['4mm', '5mm'],
 '<ROOT>': ['tweezers',
  'supply',
  'suction handle',
  'pin',
  'scissors',
  'cards',
  'multimeter',
  'arcticlean',
  'diamond impregnated files',
  'iopener',
  'paste',
  'compressed air',
  'spudger',
  'cloth',
  'pick',
  'microfiber cleaning cloth',
  'usb cable',
  'utility scissors',
  'striper',
  'cable',
  'wipes',
  'microscope',
  'handle',
  'pliers',
  'screwdriver',
  'remover',
  'programmer',
  'adhesive remover',
  'disposable gloves',
  'wire striper',
  'ipad battery isolation pick',
  'hair dryer',
  'arctic silver arcticlean',
  'awg28 wire',
  'probe_and_pickset',
  'wire',
  'isopropyl alcoho